<a href="https://colab.research.google.com/github/sranifp/PBA-Sentimen-Analisis-UBER/blob/main/Scraping_UBER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping

In [ ]:
!pip install google_play_scraper
!pip install textblob
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import app
import pandas as pd
import numpy as np
import sklearn
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import textblob
#from wordcloud import WordCloud
from pathlib import Path
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score

import pickle
import re
import time
import datetime                              # access to %%time, for timing individual notebook cells
import os
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Import seaborn styles explicitly
import seaborn as sns
# Apply the seaborn style before creating plots
sns.set_style("whitegrid")  # This line sets the Seaborn style

plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
from google_play_scraper import app, Sort, reviews_all

uber_reviews = reviews_all(
    'com.ubercab',
    sleep_milliseconds=0,
    lang='en',
    sort=Sort.NEWEST,
)

In [ ]:
df_uber = pd.DataFrame(np.array(uber_reviews),columns=['content'])
df_uber = df_uber.join(pd.DataFrame(df_uber.pop('content').tolist()))
df_uber.to_csv(r'df_uber', index=False)
from google.colab import files
files.download('df_uber')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_uber

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,76b3d3f6-6491-4539-853a-b9c69a0ea5b2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,👍🙂🙂🙂,5,0,4.601.10001,2025-10-28 09:58:48,None,NaT,4.601.10001
1,ba4c1c17-dcb9-48dd-95a8-b59ebbdaa23b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,good,5,0,4.601.10001,2025-10-28 09:55:05,None,NaT,4.601.10001
2,501420d1-a032-4981-bd64-fb948333bd2f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Doliczają dodatkowe opłaty ~10% po zakończonej...,1,0,4.601.10001,2025-10-28 09:51:39,None,NaT,4.601.10001
3,5dc1a519-a151-4c46-88df-7aae80f24a3d,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,excellent service,5,0,4.601.10001,2025-10-28 09:48:31,None,NaT,4.601.10001
4,d7f3ebd3-6e4a-41a0-8f89-380f24a59186,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,🤫🤫🤫🤫🤫🤫🤫🤫🤫🤫🤫,5,0,4.601.10001,2025-10-28 09:47:14,None,NaT,4.601.10001
...,...,...,...,...,...,...,...,...,...,...,...
1790470,05b8679c-452c-413f-bf60-7a0bdf41116b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Great service! Timely and comfortable rides. T...,5,0,1.0.9,2010-11-08 22:14:26,None,NaT,1.0.9
1790471,15db1acc-94d2-409b-bee4-70b8ebd1e865,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Sign up process is glitchy. Shoots you to the ...,3,1,1.0.9,2010-11-07 18:47:16,None,NaT,1.0.9
1790472,15a19799-0332-4f34-a49a-86d25c28fcb5,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,There's not a chance in h*ll I'd put my cc inf...,1,1,1.0.9,2010-11-05 18:52:45,None,NaT,1.0.9
1790473,05aa8694-129d-41bf-8229-e420db1f54ec,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Keeps telling me that my email and password is...,2,0,1.0.9,2010-11-04 00:29:34,None,NaT,1.0.9


In [ ]:
df_uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790475 entries, 0 to 1790474
Data columns (total 11 columns):
 #   Column                Dtype         
---  ------                -----         
 0   reviewId              object        
 1   userName              object        
 2   userImage             object        
 3   content               object        
 4   score                 int64         
 5   thumbsUpCount         int64         
 6   reviewCreatedVersion  object        
 7   at                    datetime64[ns]
 8   replyContent          object        
 9   repliedAt             datetime64[ns]
 10  appVersion            object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 150.3+ MB


In [ ]:
# Save dataframe to csv
file_name = 'uber_reviews_raw.csv'
df_uber.to_csv(file_name, index=False)